In [3]:
import pandas as pd

# import data
df1_1 = pd.read_excel('证券2.xlsx', engine='openpyxl')
df1_2 = pd.read_excel('证券1.xlsx', engine='openpyxl')
df1_3 = pd.concat([df1_1, df1_2], ignore_index=True)
df1 = df1_3[df1_3['民营指数'] >= 1]
df2 = pd.read_excel('TRD_Mnth.xlsx', engine='openpyxl')


In [7]:
df1.head()

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期
0,2013-03-31,1,57.35,1.81,31.685083,民营企业,2013-03-31
1,2013-03-31,4,36.01,0.00,inf,民营企业,2013-03-31
2,2013-03-31,5,26.93,3.64,7.398352,民营企业,2013-03-31
4,2013-03-31,7,43.38,0.00,inf,民营企业,2013-03-31
5,2013-03-31,8,69.69,0.00,inf,民营企业,2013-03-31


In [8]:
df1.dtypes

统计截止日期              object
证券代码                 int64
其他持股比例             float64
国有法人持股比例           float64
民营指数               float64
是否民营企业              object
统计日期        datetime64[ns]
dtype: object

In [6]:
df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])
df2['统计日期'] = pd.to_datetime(df2['交易月份'])

<ipython-input-6-f391f85bd79e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])


In [10]:
df1_sorted = df1.sort_values(by=['统计日期', '其他持股比例'], ascending=[True, False])
df1_sorted

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期
1167,2013-03-31,2703,94.92,0.00,inf,民营企业,2013-03-31
451,2013-03-31,981,92.33,0.60,153.883333,民营企业,2013-03-31
734,2013-03-31,2269,91.08,0.55,165.600000,民营企业,2013-03-31
1115,2013-03-31,2651,90.92,0.18,505.111111,民营企业,2013-03-31
1027,2013-03-31,2563,90.64,0.00,inf,民营企业,2013-03-31
...,...,...,...,...,...,...,...
82710,2023-12-31,2407,18.60,1.57,11.847134,民营企业,2023-12-31
154507,2023-12-31,600503,18.09,0.00,inf,民营企业,2023-12-31
154470,2023-12-31,600354,16.26,14.09,1.154010,民营企业,2023-12-31
82748,2023-12-31,2640,14.81,8.47,1.748524,民营企业,2023-12-31


In [11]:
grouped_returns = pd.DataFrame()

# 9 三季报
# 3 一季报
# 6 半年报
periods = {
    9: (1, 4),
    3: (5, 8),
    6: (9, 12)
}


periods = {
    9: (1, 4),
    3: (5, 8),
    6: (9, 12)
}

In [13]:
# 对公司进行分组 按照季度进行调仓
for year in df1['统计日期'].dt.year.unique():
    print(year)

2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [12]:

# for month, (start_month, end_month) in periods.items():
#     # 筛选出分组月份的数据
#     df1_monthly = df1_sorted[df1_sorted['统计日期'].dt.month == month]

#     # 对每一年的数据进行操作
#     for year in df1_monthly['统计日期'].dt.year.unique():
#         # 获取当前年份和上一年的数据
#         current_year_data = df1_monthly[df1_monthly['统计日期'].dt.year == year]
#         previous_year_data = df1_monthly[df1_monthly['统计日期'].dt.year == year - 1]

#         # 对当前年份的数据进行分组
#         current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')

#         # 遍历每个分组
#         for group in range(5):
#             # 获取上一年同一分组的证券代码
#             previous_group_codes = previous_year_data[previous_year_data['证券代码'].isin(current_year_data[current_year_data['分组'] == group]['证券代码'])]['证券代码']

#             # 计算加权回报率
#             for month in range(start_month, end_month + 1):
#                 # 获取对应月份的数据
#                 df2_monthly = df2[(df2['统计日期'].dt.year == year) & (df2['统计日期'].dt.month == month)]

#                 # 筛选出属于当前分组的股票
#                 df2_group = df2_monthly[df2_monthly['证券代码'].isin(previous_group_codes)]

#                 # 计算加权回报率
#                 total_market_value = df2_group['个股市值'].sum()
#                 df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
#                 weighted_return = df2_group['加权回报率'].sum()

#                 # 将结果存储到grouped_returns中
#                 new_data = pd.DataFrame({
#                     '年份': [year],
#                     '月份': [month],
#                     '分组': [group],
#                     '加权回报率': [weighted_return]
#                 })

#                 grouped_returns = pd.concat([grouped_returns, new_data], ignore_index=True)
# print(grouped_returns.head())
# grouped_returns.to_excel('计算.xlsx', index=False)

<ipython-input-12-c20ffd7fd934>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')


range = 
range(0, 5)


<ipython-input-12-c20ffd7fd934>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

range = 
range(0, 5)


<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


...

range = 
range(0, 5)


<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


range = 
range(0, 5)


<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
<ipython-input-12-c20ffd7fd934>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


     年份  月份  分组  加权回报率
0  2013   1   0    0.0
1  2013   2   0    0.0
2  2013   3   0    0.0
3  2013   4   0    0.0
4  2013   1   1    0.0
